# Post-Processing Experiment

Apply non-maximum suppression (NMS), area filtering, and Top-K selection to prune raw OWL-ViT detections down to a small set of high-quality archaeological features.

---

In [ ]:
import sys, os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().resolve().parents[2]
sys.path.insert(0, str(project_root))

import torch
import json

from tools.owlvit_utils     import OwlViTPipeline
from tools.postprocess_utils import PostprocessUtils

In [ ]:
# Experiment configuration
experiment_name      = 'postprocessing'
image_filename       = '<input_image_filename>'  # e.g., 'site_image.png'
prompts_file         = '<prompts_file>'          # e.g., 'prompts.txt'
resize_size          = (<width>, <height>)       # e.g., (1024, 1024)
thresholds           = [<threshold_1>, <threshold_2>, <threshold_3>, <threshold_4>]  # e.g., [0.001, 0.002, ...]

# Initialize pipeline
pipeline = OwlViTPipeline(
    experiment_name=experiment_name,
    resize_size=resize_size
)
pipeline.load_image(image_filename)
pipeline.load_prompts(prompts_file)


In [ ]:
# Single-shot full-image inference
inputs = pipeline.processor(
    text=pipeline.prompts,
    images=pipeline.image_resized,
    return_tensors="pt"
)
with torch.no_grad():
    outputs = pipeline.model(**inputs)

for thresh in thresholds:
    raw_results = pipeline.processor.post_process_object_detection(
        outputs,
        target_sizes=torch.tensor([[pipeline.image_full.height,
                                    pipeline.image_full.width]]),
        threshold=thresh
    )[0]
    
    print(f"--- Threshold = {thresh:.4f} ---")

    print(f"Raw detections at t={thresh}: {len(raw_results['scores'])}")
    
    # A) Filter to a specific prompt
    target_prompt = "<single_prompt>"
    filt1 = PostprocessUtils.filter_by_prompt(
        raw_results,
        prompt=target_prompt,
        prompts=pipeline.prompts
    )
    print("After prompt filter:", len(filt1['scores']))

    # B) NMS to collapse overlaps
    nmsed = PostprocessUtils.apply_nms(filt1, iou_threshold=0.3)
    print("After NMS:", len(nmsed['scores']))

    # C) Area-based filter (0.005%–5% of image area)
    areaed = PostprocessUtils.area_filter(
        nmsed,
        min_pct=0.00005,
        max_pct=0.05,
        image_size=(pipeline.image_full.width, pipeline.image_full.height)
    )
    print("After area filter:", len(areaed['scores']))

    # D) Top-K to pick the strongest boxes (e.g. K=12)
    final = PostprocessUtils.top_k(areaed, k=12)
    print("After top-K:", len(final['scores']))
    
        # Visual check
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(6,6))
    ax.imshow(final)
    ax.axis('off')
    plt.show()

    # Override pipeline.image_resized with preprocessed image
    pipeline.image_resized = final.resize(pipeline.resize_size)
    
    # Save the overlaid detection PNG
    png_path = pipeline.save_visualisation(
            final,
            threshold=thresh
            
    ) 
    
    # Save the per-threshold metrics JSON
    json_path = pipeline.save_metrics(
        results,
        threshold=thresh,
        suffix='tiling'
    )
    print("Saved metrics to:", json_path)
    
    # save geojson
    geojson_path = pipeline.run_and_save_geojson(thresh)
